# Chat-Bot Program
Marina MOESLE

## ReadMe / Info

I have encountered many problems with "multiple instance of bot", errors saying that my bot was opened in multiple session. 

At first, I tried to deal with updating information about gas price with class, and a Handle function depanding on user_id, but with mutliple different way (using getUpdate, using ChatHandler, using a list saving the user_id...) it stills leads to problem saying that multiple instance of my bot were open. 
That is why I have choosen to make it differently with only a schedul, and global variables.

To deal with this problem, I have searched and found that there is 2 principal solutions : 
   - clean-up my python script's session with the command "taskkill /IM python.exe /F" in my terminal,
   
   - or force the stop of my bot (precisely his WebHook) with an API URL, whiches are (filled-up with my bot's Token):

https://api.telegram.org/bot8018674828:AAHA1lyYDxnOo5V0F9xllSSK1LnvFJG4xi4/deleteWebhook : to delete any WebHook that could be launched
https://api.telegram.org/bot8018674828:AAHA1lyYDxnOo5V0F9xllSSK1LnvFJG4xi4/getWebhookInfo : to check if it has been closed
https://api.telegram.org/bot8018674828:AAHA1lyYDxnOo5V0F9xllSSK1LnvFJG4xi4/getUpdates?offset=-1 : to reinitialise my bot reading my messages
        
Therefore now, I only start once my bot and reuse it in other function and make sure any schedul is closed before launching a new for my bot.                                             

### Part A

In [1]:
# installations and libraries
#pip install apscheduler
import requests
import json
from apscheduler.schedulers.background import BackgroundScheduler
import time

In [2]:
#%% importation of HTML page with the first request : Get Estimation of Confirmation Time

parameters = {'key1' : 'gasprice'}
r1 = requests.get('https://api.etherscan.io/api?module=gastracker'
                 +'&action=gasestimate'
                 +'&gasprice=2000000000'
                 +'&apikey=censorship_of_my_APIKey',
                 params=parameters
                 )
# It returned 45 seconds

In [3]:
r1.text

'{"status":"1","message":"OK","result":"45"}'

In [20]:
#%% part "Get Gas Oracle"

r = requests.get('https://api.etherscan.io/api'
                 +'?module=gastracker'
                 +'&action=gasoracle'
                 +'&apikey=censorship_of_my_APIKey')


In [23]:
#%% converts the request via json and recolting the result dictionnary to obtain FastGasPrice and gasUsedRatio
def Get_GasInfo():
    r = requests.get('https://api.etherscan.io/api'
                 +'?module=gastracker'
                 +'&action=gasoracle'
                 +'&apikey=censorship_of_my_APIKey')
    try :
        gas_info = r.json()['result']
        fgp = float(gas_info['FastGasPrice'])
        gur = float(gas_info['gasUsedRatio'].split(',')[0]) #to obtain only the first value
        print(f'FastGasPrice: {fgp:.2f}, gasUsedRatio: {gur:.2f}')
    except requests.exceptions.JSONDecodeError:
        print('Error during finding the content. (204)')
    
#%% doing the analysis every 2 minutes
schedul = BackgroundScheduler()
schedul.start()
schedul.add_job(Get_GasInfo, 'interval', minutes = 2, id='update_gas_info')

try:
    while True:
        time.sleep(1)  
except KeyboardInterrupt:
    schedul.remove_job('update_gas_info')
    schedul.shutdown()

FastGasPrice: 0.51, gasUsedRatio: 0.26
FastGasPrice: 0.51, gasUsedRatio: 0.79
FastGasPrice: 0.53, gasUsedRatio: 0.44


### Part B

In [25]:
pip install telepot --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import telepot.helper
from telepot.loop import MessageLoop
from telepot.delegate import per_chat_id, create_open, pave_event_space

In [3]:
bot = telepot.Bot('8018674828:AAHA1lyYDxnOo5V0F9xllSSK1LnvFJG4xi4')
print(bot.getMe())

{'id': 8018674828, 'is_bot': True, 'first_name': 'GazouilleBot', 'username': 'GazouilleurBot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False, 'can_connect_to_business': False, 'has_main_web_app': False}


In [4]:
bot.sendMessage(8187409056, 'Howdy, I am GazouilleBot, here for your daily gas price update !')

{'message_id': 313,
 'from': {'id': 8018674828,
  'is_bot': True,
  'first_name': 'GazouilleBot',
  'username': 'GazouilleurBot'},
 'chat': {'id': 8187409056,
  'first_name': 'Marina',
  'last_name': 'MOESLE',
  'type': 'private'},
 'date': 1743684759,
 'text': 'Howdy, I am GazouilleBot, here for your daily gas price update !'}

And now my bot sending his update informations about gas price every two minutes : 

In [5]:
# to stock the previous value, I put it outside the function
previous_value = None
my_userID = 8187409056

def GazouilleBotMission():
    global previous_value
    global my_userID
    
    # same as get gas info function
    r = requests.get('https://api.etherscan.io/api'
                 +'?module=gastracker'
                 +'&action=gasoracle'
                 +'&apikey=censorship_of_my_APIKey')
    try :
        gas_info = r.json()['result']
        fgp = float(gas_info['FastGasPrice'])
        gur = float(gas_info['gasUsedRatio'].split(',')[0]) #to obtain only the first value
        change = fgp - previous_value if previous_value is not None else 0.0
        bot.sendMessage(my_userID,f'FastGasPrice : {fgp:.2f}, GasUsedRatio : {gur:.2f}, Change : {change:.2f}')
        previous_value = fgp
    except requests.exceptions.JSONDecodeError:
        bot.sendMessage(my_userID,'I have noticed some error during finding the content... (204) :/')
    
    
def handle(msg):
    global my_userID
    global last_update_id
    global sched

    chat_id = msg['chat']['id']
    text = msg.get('text', '').lower()

    greetings = ['hey', 'hello', 'coucou', 'bonjour', 'good morning']
    
    if text in greetings:
        bot.sendMessage(chat_id, "Howdy, I am GazouilleBot, here for your daily gas price update -_(^u ^)-_ !")
        
    elif text == "stop":
        bot.sendMessage(chat_id, "Goodbye ! ")
        schedul_bot.pause()

schedul_bot = BackgroundScheduler()
schedul_bot.start()   
MessageLoop(bot, handle).run_as_thread()
schedul_bot.add_job(GazouilleBotMission, 'interval', minutes = 2, id='gazouille_bot_mission')

try:
    while True:
        time.sleep(1)  
except KeyboardInterrupt:
    schedul_bot.shutdown()

## Part C

This was my first approache : I add a line to my code so that when user says "/restart", it relaunche the schedul_bot

In [4]:
# to stock the previous value, I put it outside the function
previous_value = None
my_userID = 8187409056
updates_running = True

def GazouilleBotMission():
    global previous_value
    global my_userID
    global updates_running
    
    if not updates_running:
        return 
    
    # same as get gas info function
    r = requests.get('https://api.etherscan.io/api'
                 +'?module=gastracker'
                 +'&action=gasoracle'
                 +'&apikey=censorship_of_my_APIKey')
    try :
        gas_info = r.json()['result']
        fgp = float(gas_info['FastGasPrice'])
        gur = float(gas_info['gasUsedRatio'].split(',')[0]) #to obtain only the first value
        change = fgp - previous_value if previous_value is not None else 0.0
        bot.sendMessage(my_userID,f'FastGasPrice : {fgp:.2f}, GasUsedRatio : {gur:.2f}, Change : {change:.2f}')
        previous_value = fgp
    except requests.exceptions.JSONDecodeError:
        bot.sendMessage(my_userID,'I have noticed some error during finding the content... (204) :/')
    
    
def handle(msg):
    global my_userID
    global updates_running
    global previous_value
    
    text = msg.get('text', '').lower()

    greetings = ['hey', 'hello', 'coucou', 'bonjour', 'good morning']
    
    if text in greetings:
        bot.sendMessage(my_userID, "Howdy, I am GazouilleBot, here for your daily gas price update -_(^u ^)-_ !")
        
    elif text == "stop":
        bot.sendMessage(my_userID, "Goodbye ! ")
        updates_running = False
    
    elif text == '/restart':
        if not updates_running :
            updates_running = True
        previous_value = None
        bot.sendMessage(my_userID, "Let's start again -_(^u ^)-_ !")
            
schedul_bot = BackgroundScheduler()
schedul_bot.start()
MessageLoop(bot, handle).run_as_thread()
schedul_bot.add_job(GazouilleBotMission, 'interval', minutes = 2, id='gazouille_bot_mission')

try:
    while True:
        time.sleep(1)  
except KeyboardInterrupt:
    schedul_bot.shutdown()

## Video link : 

https://youtu.be/Tq1XaVF7pHA 